In [ ]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
#모델 불러오기
import joblib
model2 = joblib.load('best_model.pkl')

In [ ]:
while True:
    try:
        print('1.태블릿 PC 시세 검색 2.기종 추천 받기')
        select = int(input('원하는 기능을 선택하세요: '))
        if (select!=1 and select!=2):
            raise Exception
        break
    except Exception:
        print("잘못된 값입니다. 다시 입력하세요.\n")

In [ ]:
#출시일 dictionary
date_dic = {'normal2':2011.04,'normal3':2012.03,'normal4':2012.10,'normal5':2017.03,
         'normal6':2018.03,'normal7':2019.09,'normal8':2020.09,'mini1':2012.11,
         'mini2':2013.10,'mini3':2014.10,'mini4':2015.09,'air1':2013.10,'air2':2014.10,
         'air3':2019.03,'air4':2020.09,'pro1':2016.03,'pro2':2017.06,'pro3':2018.10,'pro4':2020.03}

#name
name_dic = {1:'air',2:'mini',3:'normal',4:'pro'}

In [ ]:
class net(torch.nn.Module):
    def __init__(self, base_model, base_out_features, num_classes):
        super(net,self).__init__()
        self.base_model=base_model
        self.linear1 = torch.nn.Linear(base_out_features, 512)
        self.output = torch.nn.Linear(512,num_classes)
    def forward(self,x):
        x = F.relu(self.base_model(x))
        x = F.relu(self.linear1(x))
        x = self.output(x)
        return x

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

In [ ]:
res = torchvision.models.resnet50(pretrained=True)
for param in res.parameters():
    param.requires_grad=False

model = net(base_model=res, base_out_features=res.fc.out_features, num_classes=3)
model = model.to(device)

In [ ]:
model.load_state_dict(torch.load('./best_model.pth', map_location=device))
model.eval()

In [ ]:
class setting(Dataset):
    def __init__(self,dataframe,transform):
        self.dataframe = dataframe
        self.transform = transform
        
    def __getitem__(self,index):
        x = Image.open(self.dataframe.iloc[index,0]).convert('RGB')
        x = self.transform(x)
        return x
        
    def __len__(self):
        return self.dataframe.shape[0]

In [ ]:
def testing(test_loader,model):
  output = []

  for x in test_loader:
    x = x.to(device)
    y = model(x)
    _,predicted = torch.max(y.data, 1)
    predicted = predicted.cpu()
    index = predicted.numpy()[0]
    output.append(index)
  
  return output

In [ ]:
import pandas as pd
if(select ==1):
    while True:
        try:
            date = float(input('판매할 날짜를 입력하세요(년도.월): '))
            break
        except Exception:
            print('잘못된 값입니다. 다시 입력하세요.\n')
    #type입력
    while True:
        try:
            type_ = int(input('태블릿PC의 type을 선택하세요 (1.air 2.mini 3.normal 4.pro): '))
            if(type_!=1 and type_!=2 and type_!=3 and type_!=4):
                raise Exception
            break
        except Exception:
            print('잘못된 값입니다. 다시 입력하세요.\n')
    #generation 입력
    while True:
        try:
            age = int(input('세대를 입력하세요: '))
            break
        except Exception:
            print('잘못된 값입니다. 다시 입력하세요.\n')
            
    #state 맞추기
    while True:
        try:
            img = input("이미지 파일 경로 입력하세요: ")
          
            break
        except Exception:
            print('잘못된 값입니다. 다시 입력하세요.\n')
    #capacity
    while True:
        try:
            capacity = int(input('용량을 입력하세요: '))
            break
        except Exception:
            print('잘못된 값입니다. 다시 입력하세요.\n')
    #wifi
    while True:
        try:
            wifi = int(input('Connectivity를 선택하세요 (1.lte 2.wifi): '))
            if(wifi!=1 and wifi!=2):
                raise Exception
            break
        except Exception:
            print('잘못된 값입니다. 다시 입력하세요.\n')
    
    #출시날짜 추출
    name = name_dic[type_]+str(age)
    date_i = date_dic[name]
    used = date-date_i
    
     #시세 검색
    
    #판매날짜
    #상태 A=0,B=1,C=2
    #wifi=1, lte =0
    #air =0, mini=1, normal =2, pro=3
    wifi=wifi-1
    type_=type_-1

     #상태추출               
    test_img = {'img':[img]}
    test_img = pd.DataFrame(test_img)
    transformer = transforms.Compose([transforms.Resize(256),
                                  transforms.CenterCrop(224),
                                  transforms.ToTensor(),
                                  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    test_dataset = setting(test_img,transform=transformer)
    test_loader = DataLoader(test_dataset)
    state=testing(test_loader,model)[0]

    if (state==0):
      state=1
    elif (state==1):
      state=0

    x_data = pd.DataFrame([[state,date,wifi,used,capacity,type_,age]])
    y =model2.predict(x_data)
    print('\n\n')
    print('ipad',name,'의 시세는',int(y),'원 입니다.')

elif (select==2):
    #기종추천  
    while True:
        try:
            min = int(input('최소 가격을 입력하세요 (단위:만원): '))
            break
        except Exception:
            print('잘못된 값입니다. 다시 입력하세요.\n')
    while True:
        try:
            max = int(input('최대 가격을 입력하세요 (단위:만원): '))
            if(min>max):
                raise Exception
            break
        except Exception:
            print('잘못된 값입니다. 다시 입력하세요.\n')
    
    #상태선택
    while True:
        try:
            state_w = int(input('원하는 상태를 선택하세요 (1.미개봉 2.거의 새 것 3.사용감 있음): '))
            if(state_w!=1 and state_w!=2 and state_w!=3):
                raise Exception
            break
        except Exception:
            print('잘못된 값입니다. 다시 입력하세요.\n')
    
    #wifi선택
    while True:
        try:
            wifi_w = int(input('원하는 Connectivity를 선택하세요 (1.lte 2.wifi): '))
            if(wifi_w!=1 and wifi_w!=2):
                raise Exception
            break
        except Exception:
            print('잘못된 값입니다. 다시 입력하세요.\n')
            
    #현재날까 불러오기
    from datetime import datetime
    year=datetime.today().strftime("%Y")
    month=datetime.today().strftime("%m")
    today = year+'.'+month
    today=float(today)
    
    #used 계산
    u_dic={}
    for key, value in date_dic.items():
        u_dic[key]=round(today-value,2)
   
    state_list=[0,1,2]
    wifi_list=[0,1]
    capacity_list=[32,64,128,256,512,1024]
    name_list=list(u_dic.keys())
    
    #파이썬 list 모든 조합 구하기
    from itertools import product
    cx=pd.DataFrame(product(state_list,wifi_list,capacity_list,name_list))
    cx.columns=['state','wifi','capacity','name']
    cx['used']=0.0
    cx['date']=today
    cx['age']=0
    cx['type']=0

    #실제와 다른 행 지우기
    idx = cx[((cx['name']=='normal2')&(cx['capacity']!=16)&(cx['capacity']!=32)&(cx['capacity']!=64))
        |((cx['name']=='normal3')&(cx['capacity']!=32)&(cx['capacity']!=64))
        |((cx['name']=='normal4')&(cx['capacity']!=16)&(cx['capacity']!=32)&(cx['capacity']!=64))
        |((cx['name']=='normal5')&(cx['capacity']!=32)&(cx['capacity']!=128))
        |((cx['name']=='normal6')&(cx['capacity']!=32)&(cx['capacity']!=128))
        |((cx['name']=='normal7')&(cx['capacity']!=32)&(cx['capacity']!=128))
        |((cx['name']=='normal8')&(cx['capacity']!=32)&(cx['capacity']!=128))
        |((cx['name']=='pro1')&(cx['capacity']!=32)&(cx['capacity']!=128)&(cx['capacity']!=256))
        |((cx['name']=='pro2')&(cx['capacity']!=64)&(cx['capacity']!=256)&(cx['capacity']!=512))
        |((cx['name']=='pro3')&(cx['capacity']!=64)&(cx['capacity']!=256)&(cx['capacity']!=512)&(cx['capacity']!=1024))
        |((cx['name']=='pro4')&(cx['capacity']!=128)&(cx['capacity']!=256)&(cx['capacity']!=512)&(cx['capacity']!=1024))
        |((cx['name']=='air1')&(cx['capacity']!=16)&(cx['capacity']!=32)&(cx['capacity']!=64)&(cx['capacity']!=128))
        |((cx['name']=='air2')&(cx['capacity']!=16)&(cx['capacity']!=64)&(cx['capacity']!=128))
        |((cx['name']=='air3')&(cx['capacity']!=64)&(cx['capacity']!=256))
        |((cx['name']=='air4')&(cx['capacity']!=64)&(cx['capacity']!=256))
        |((cx['name']=='mini1')&(cx['capacity']!=16)&(cx['capacity']!=32)&(cx['capacity']!=64))
        |((cx['name']=='mini2')&(cx['capacity']!=16)&(cx['capacity']!=32)&(cx['capacity']!=64)&(cx['capacity']!=128))
        |((cx['name']=='mini3')&(cx['capacity']!=16)&(cx['capacity']!=64)&(cx['capacity']!=128))
        |((cx['name']=='mini4')&(cx['capacity']!=16)&(cx['capacity']!=64)&(cx['capacity']!=128))].index
    cx = cx.drop(idx,axis=0)

    cx=cx.reset_index()
    #used 구하기
    for i in range(len(cx)):
      cx['used'][i]=u_dic[cx['name'][i]]
      temp = cx['name'][i]
      number = temp[-1:] #맨끝숫자
    
      temp = temp[:-1] #숫자 제거
      if(temp=='air'):
         cx['type'][i]=0
      elif(temp=='mini'):
         cx['type'][i]=1
      elif(temp=='normal'):
          cx['type'][i]=2
      elif(temp=='pro'):
          cx['type'][i]=3
      cx['age'][i]=number
    cx = cx[['state','date','wifi','used','capacity','type','age']]

    result =model2.predict(cx)

    cx['y']=0
    for i in range(len(result)):
      cx['y'][i] = round(result[i],0)

    state_w= state_w-1
    wifi_w=wifi_w-1
    min=min*10000
    max=max*10000

    if (wifi_w==0):
      wifi_p='lte'
    elif (wifi_w==1):
     wifi_p='wifi'
    
    if (state_w==0):
     state_p='A'
    elif (state_w==1):
     state_p='B'
    elif (state_w==2):
     state_p='C'

     #범위에 맞는 값 찾아서 출력해주기
    answer_=cx[(cx['y']>=min)&(cx['y']<=max)&(cx['state']==state_w)&(cx['wifi']==wifi_w)]
    answer_=answer_.reset_index()
    print('================추천 모델==================')
    print("%11s"%"model"+"%10s"%"capacity"+"%6s"%"wifi"+"%7s"%"state"+"%7s"%"price")
    print('===========================================')
    for i in range(len(answer_)):
      if(answer_['type'][i]==0):
         model_name='air'
      elif(answer_['type'][i]==1):
         model_name='mini'
      elif(answer_['type'][i]==2):
          model_name='ipad'
      elif(answer_['type'][i]==3):
          model_name='pro'
        
      model_name=model_name+str(cx['age'][i])
      print("%-6s"%("["+str(i+1)+"]")+"%-9s"%model_name+"%-8s"%(str(answer_['capacity'][i])+"GB ")+"%-8s"%(wifi_p)+"%-4s"%(state_p)+str(answer_['y'][i])+"원")